### Projet IGL3/IDS3 : Analyse Numérique Matricielle dans le cadre de la gestion de bases de données bibliographiques 

In [ ]:
import numpy as np
from fuzzywuzzy import fuzz

In [ ]:
# 1ère Partie : Calcul de score de document : Sans utilisation de la décomposition SVD
# Modèle de l'espace vectoriel

def matrice_termes_documents(termes, documents):
    """
      Cette méthode permet de construire la matrice termes-documents.
    """
    matrice = np.zeros((len(termes), len(documents)), dtype=int)

    termes_sorted = sorted(termes)
    termes_sorted = [terme.lower() for terme in termes_sorted]

    documents_termes = []

    for doc in documents :
        if isinstance(doc, str) :
            termes_doc = doc.lower().split()
        else :
            termes_doc = doc

        documents_termes.append(termes_doc)

    for i , terme in enumerate(termes_sorted) :
        for j, doc_termes in enumerate(documents_termes) :
            if terme in doc_termes :
                matrice[i][j] = 1

    return matrice 

In [ ]:
def afficher_matrice(matrice) :
    """
      Cette méthode permet d'afficher la matrice termes-documents.
    """
    for i in range(matrice.shape[0]) :
        for j in range(matrice.shape[1]) :
            print(matrice[i][j], end = " ")
        print()

In [ ]:
#Test de la méthode : 
# Génération de la matrice termes-documents (Exemple 1)
documents = [
    "Algèbre linéaire et matrices",
    "Analyse réele et suites",
    "Probabilités et statistiques",
    "Matrices et déterminants"
]

termes = [
    "algèbre",
    "matrices",
    "analyse",
    "suites",
    "probabilités",
    "statistiques",
    "déterminants"
]


matrice = matrice_termes_documents(termes, documents)
afficher_matrice(matrice)

In [ ]:
from fuzzywuzzy import fuzz
import numpy as np

def requete_vers_colonne(termes, requete):
    vecteur = [0] * len(termes) 
    termes_sorted = sorted(termes)
    mots_requete = requete.lower().split()
    
    for i, terme in enumerate(termes_sorted):
        for mot in mots_requete:
            if fuzz.ratio(terme.lower(), mot) >= 90:
                vecteur[i] = 1  
                break  
    
    return vecteur


In [ ]:
#Test de la méthode :
requete = "Algèbre matrice"
vecteur_colonne = requete_vers_colonne(termes, requete)
vecteur_colonne

In [ ]:
def score_document(j , matrice , q):
    """
      Cette méthode permet de calculer le score d'un document j par rapport à une requête q.
    """
    vect_doc = matrice[:, j]
    norme_req = np.linalg.norm(q)
    if norme_req == 0:
        return 0
    
    norme_doc = np.linalg.norm(vect_doc)
    if norme_doc == 0:
        return 0
    
    produit_scalaire = np.dot(vect_doc, q)

    score =  float(produit_scalaire / (norme_doc * norme_req))
    return round(score, 2)

In [ ]:
# Affichage des scores de documents : 
def tab_score(matrice , q):
    """
      Cette méthode permet de calculer le score de chaque document par rapport à la requête q.
    """
    scores = []
    for j in range(matrice.shape[1]):
        score = score_document(j, matrice, q)
        scores.append(score)
    return scores

In [ ]:
def resultat_requete(matrice, q , score_min = 0.8):
    """
    Cette méthode permet d'afficher le score de chaque document par rapport à la requête q
    et détermine le document le plus pertinent.
    """
    scores = tab_score(matrice, q)
    
    for i, score in enumerate(scores):
        print(f"Document {i+1} : {score}")
    
    if len(scores) > 0 and max(scores) > score_min :
        indice_max = scores.index(max(scores))
        print(f"\nLe document le plus pertinent est le Document {indice_max+1} avec un score de {scores[indice_max]}")
    else:
        print("\nAucun document disponible pour évaluation")

In [ ]:
#Test des méthodes (Sans utilisation de la décomposition SVD) :
# Exemple 1 : 

requete = "Algèbre matrice"
vecteur_colonne = requete_vers_colonne(termes, requete)

print("---------------AFFICHAGE DE LA MATRICE---------------\n")
matrice = matrice_termes_documents(termes, documents)
afficher_matrice(matrice)
print("\n---------------TRAVAIL DE LA REQUETE---------------\n")
# Affichage des scores de documents :
resultat_requete(matrice, vecteur_colonne, score_min = 0.8)


In [ ]:
# Avant le test de l'exemple 3 : Génération de la matrice termes-documents (Exemple 3)
documents_exemple_3 = [
    "Croissance PIB Investissement",
    "Inflation Monnaie Dépression",
    "Commerce Exportation Croissance",
    "Emploi Chômage Salaire",
    "Impôts Fiscalité Budget",
    "Géologie Faille Tremblement",
    "Volcan Séisme Plaque tectonique",
    "Dépression Bassin Erosion",
    "Stratigraphie Couches Roche",
    "Gisement Forage Bassin"
]

termes_exemple_3 = [
    "Bassin",
    "Chômage",
    "Croissance",
    "Dépression",
    "Fiscalité",
    "Séisme"
]

matrice_exemple_3 = matrice_termes_documents(termes_exemple_3, documents_exemple_3)
afficher_matrice(matrice_exemple_3)

In [ ]:
# Exemple 3 (REQUERTES) :
requete_exemple_3_1 = "dépression croissance"
requete_exemple_3_2 = "bassin fiscalité"

In [ ]:
#Test de la méthode (EXEMPLE 3) :
vecteur_colonne_exemple_3_1 = requete_vers_colonne(termes_exemple_3, requete_exemple_3_1)
vecteur_colonne_exemple_3_2 = requete_vers_colonne(termes_exemple_3, requete_exemple_3_2)

print("\n---------------TRAVAIL DE LA REQUETE 1---------------\n")
resultat_requete(matrice_exemple_3, vecteur_colonne_exemple_3_1, score_min = 0.8)


In [ ]:
print("\n---------------TRAVAIL DE LA REQUETE 2---------------\n")
resultat_requete(matrice_exemple_3, vecteur_colonne_exemple_3_1, score_min = 0.8)